In [ ]:
#Mount Google Drive in Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load CSVs into DuckDB ( free “warehouse”)

!pip -q install duckdb

import duckdb
import pandas as pd

base = "/content/drive/MyDrive/DI-Agent-AutoParts"

con = duckdb.connect()

con.execute(f"CREATE TABLE dim_sku AS SELECT * FROM read_csv_auto('{base}/dim_sku.csv')")
con.execute(f"CREATE TABLE dim_calendar AS SELECT * FROM read_csv_auto('{base}/dim_calendar.csv')")
con.execute(f"CREATE TABLE fact_sales AS SELECT * FROM read_csv_auto('{base}/fact_sales.csv')")
con.execute(f"CREATE TABLE fact_inventory AS SELECT * FROM read_csv_auto('{base}/fact_inventory.csv')")
con.execute(f"CREATE TABLE fact_forecast AS SELECT * FROM read_csv_auto('{base}/fact_forecast.csv')")
con.execute(f"CREATE TABLE fact_costs AS SELECT * FROM read_csv_auto('{base}/fact_costs.csv')")

con.execute("SHOW TABLES").fetchdf()


,name
0,dim_calendar
1,dim_sku
2,fact_costs
3,fact_forecast
4,fact_inventory
5,fact_sales


In [ ]:
con.execute("SHOW TABLES").fetchdf()

,name
0,dim_calendar
1,dim_sku
2,fact_costs
3,fact_forecast
4,fact_inventory
5,fact_sales


In [ ]:
con.execute("""
SELECT 'dim_sku' AS table, COUNT(*) AS rows FROM dim_sku
UNION ALL SELECT 'dim_calendar', COUNT(*) FROM dim_calendar
UNION ALL SELECT 'fact_sales', COUNT(*) FROM fact_sales
UNION ALL SELECT 'fact_inventory', COUNT(*) FROM fact_inventory
UNION ALL SELECT 'fact_forecast', COUNT(*) FROM fact_forecast
UNION ALL SELECT 'fact_costs', COUNT(*) FROM fact_costs
""").fetchdf()


,table,rows
0,dim_sku,40
1,dim_calendar,184
2,fact_sales,13248
3,fact_inventory,9200
4,fact_forecast,1072
5,fact_costs,2760


In [ ]:
# KPI Monitors weekly KPIs:

Revenue ($)

Inventory value ($)

Forecast error (abs %)

SyntaxError: invalid syntax (ipython-input-712110261.py, line 3)

In [ ]:
kpi_weekly = con.execute("""
WITH sales_w AS (
  SELECT
    year(date) AS year,
    week(date) AS week,
    SUM(revenue) AS revenue
  FROM fact_sales
  GROUP BY 1,2
),
inv_w AS (
  SELECT
    year(date) AS year,
    week(date) AS week,
    SUM(inventory_value) AS inventory_value
  FROM fact_inventory
  GROUP BY 1,2
),
fc_w AS (
  SELECT
    year,
    iso_week AS week,
    AVG(ABS(forecast_error_pct)) AS abs_forecast_error_pct
  FROM fact_forecast
  GROUP BY 1,2
)
SELECT
  s.year, s.week,
  ROUND(s.revenue, 2) AS revenue,
  ROUND(i.inventory_value, 2) AS inventory_value,
  ROUND(f.abs_forecast_error_pct, 2) AS abs_forecast_error_pct
FROM sales_w s
LEFT JOIN inv_w i USING (year, week)
LEFT JOIN fc_w f USING (year, week)
ORDER BY s.year, s.week;
""").fetchdf()

kpi_weekly.tail(20)


,year,week,revenue,inventory_value,abs_forecast_error_pct
7,2025,33,161395.05,11705421.28,7.60
8,2025,34,153610.34,10959874.57,6.81
9,2025,35,153057.95,11654894.00,6.90
10,2025,36,160724.22,10911921.12,6.96
11,2025,37,157262.72,11235618.43,8.85
12,2025,38,160656.00,10587077.27,7.78
13,2025,39,157175.60,11645572.06,8.70
14,2025,40,180898.80,12254163.97,8.40
15,2025,41,187485.05,10914233.65,11.25
16,2025,42,185058.15,11665268.20,10.48


In [ ]:
  #Anomaly Detector Is anything unusual happening this week compared to normal behavior?

  #Flags weeks where revenue or inventory is unusually high/low.

  #Iimplemented a lightweight anomaly detection layer using z-scores to flag weeks where revenue
  #or inventory deviated materially from historical norms. These flags triggered root-cause analysis and automated executive summaries

  import pandas as pd
#using z score - How far is this value from normal, measured in standard deviations formula = z= (value - mean)/standard deviation
#Business translation z=0 ~ normal; z>2 ~ unusual ; z>=3 extreme
#add_zscore_flags() function This function: calculates z-scores; flags anomalies; returns a new DataFrame

def add_zscore_flags(df, col, z=2.0):
    mu = df[col].mean()
    sigma = df[col].std(ddof=0) if df[col].std(ddof=0) != 0 else 1.0
    out = df.copy()
    out[f"{col}_z"] = (out[col] - mu) / sigma
    out[f"{col}_flag"] = out[f"{col}_z"].abs() >= z
    return out

flags = add_zscore_flags(kpi_weekly, "revenue", z=2.0)
flags = add_zscore_flags(flags, "inventory_value", z=2.0)

flagged_weeks = flags[flags["revenue_flag"] | flags["inventory_value_flag"]].sort_values(["year","week"])
flagged_weeks


,year,week,revenue,inventory_value,abs_forecast_error_pct,revenue_z,revenue_flag,inventory_value_z,inventory_value_flag
0,2025,1,69973.19,5419929.79,7.26,-4.135847,True,-4.579901,True


In [ ]:
#If flagged_weeks comes back empty, lower the threshold:
flags = add_zscore_flags(kpi_weekly, "revenue", z=1.5)
flags = add_zscore_flags(flags, "inventory_value", z=1.5)
flagged_weeks = flags[flags["revenue_flag"] | flags["inventory_value_flag"]].sort_values(["year","week"]) #Show me weeks where either revenue or inventory is abnormal.
flagged_weeks


,year,week,revenue,inventory_value,abs_forecast_error_pct,revenue_z,revenue_flag,inventory_value_z,inventory_value_flag
0,2025,1,69973.19,5419929.79,7.26,-4.135847,True,-4.579901,True


In [ ]:
# Root Cause (Top Revenue + Inventory Drivers)
YEAR = int(flagged_weeks.iloc[-1]["year"])
WEEK = int(flagged_weeks.iloc[-1]["week"])
YEAR, WEEK

#A) Revenue drivers (category + region delta vs prior week)
rev_drivers = con.execute(f"""
WITH cur AS (
  SELECT product_category, region, SUM(revenue) AS rev
  FROM fact_sales
  WHERE year(date)={YEAR} AND week(date)={WEEK}
  GROUP BY 1,2
),
prev AS (
  SELECT product_category, region, SUM(revenue) AS rev
  FROM fact_sales
  WHERE year(date)={YEAR} AND week(date)={WEEK-1}
  GROUP BY 1,2
)
SELECT
  COALESCE(c.product_category,p.product_category) AS product_category,
  COALESCE(c.region,p.region) AS region,
  ROUND(COALESCE(c.rev,0),2) AS rev_cur,
  ROUND(COALESCE(p.rev,0),2) AS rev_prev,
  ROUND(COALESCE(c.rev,0) - COALESCE(p.rev,0),2) AS rev_delta
FROM cur c
FULL OUTER JOIN prev p
  ON c.product_category=p.product_category AND c.region=p.region
ORDER BY ABS(rev_delta) DESC
LIMIT 10;
""").fetchdf()

rev_drivers

,product_category,region,rev_cur,rev_prev,rev_delta
0,Rotors,Northeast,4951.19,0.0,4951.19
1,Rotors,South,4776.29,0.0,4776.29
2,Rotors,Midwest,4406.73,0.0,4406.73
3,Rotors,West,4268.26,0.0,4268.26
4,Struts,South,2764.27,0.0,2764.27
5,Control Arms,West,2567.36,0.0,2567.36
6,Control Arms,South,2455.47,0.0,2455.47
7,Brake Pads,South,2386.96,0.0,2386.96
8,Brake Pads,Midwest,2216.07,0.0,2216.07
9,Struts,Midwest,2137.81,0.0,2137.81


In [ ]:
# B) Inventory drivers (SKU + location delta vs prior week)
inv_drivers = con.execute(f"""
WITH cur AS (
  SELECT sku, location, SUM(inventory_value) AS inv
  FROM fact_inventory
  WHERE year(date)={YEAR} AND week(date)={WEEK}
  GROUP BY 1,2
),
prev AS (
  SELECT sku, location, SUM(inventory_value) AS inv
  FROM fact_inventory
  WHERE year(date)={YEAR} AND week(date)={WEEK-1}
  GROUP BY 1,2
)
SELECT
  COALESCE(c.sku,p.sku) AS sku,
  COALESCE(c.location,p.location) AS location,
  ROUND(COALESCE(c.inv,0),2) AS inv_cur,
  ROUND(COALESCE(p.inv,0),2) AS inv_prev,
  ROUND(COALESCE(c.inv,0) - COALESCE(p.inv,0),2) AS inv_delta
FROM cur c
FULL OUTER JOIN prev p
  ON c.sku=p.sku AND c.location=p.location
ORDER BY ABS(inv_delta) DESC
LIMIT 10;
""").fetchdf()

inv_drivers

,sku,location,inv_cur,inv_prev,inv_delta
0,SUST006,PHL-DC,347452.84,0.0,347452.84
1,SUST001,ATL-DC,254442.52,0.0,254442.52
2,SUST006,DAL-DC,204393.87,0.0,204393.87
3,BRRO001,LAX-DC,181114.56,0.0,181114.56
4,SUST006,LAX-DC,146549.60,0.0,146549.60
5,SUST001,CHI-DC,132182.07,0.0,132182.07
6,SUST002,CHI-DC,129009.60,0.0,129009.60
7,BRRO004,DAL-DC,125553.58,0.0,125553.58
8,LIHE004,CHI-DC,111756.11,0.0,111756.11
9,SUST006,ATL-DC,109766.90,0.0,109766.90


In [ ]:
#Executive Narrative (rule-based “agent output”)

import pandas as pd

def pct_change(cur, prev):
    return None if prev == 0 else (cur - prev) / prev * 100.0

kpi_weekly_sorted = kpi_weekly.sort_values(["year", "week"]).reset_index(drop=True)

# Locate current week
mask_cur = (kpi_weekly_sorted["year"] == YEAR) & (kpi_weekly_sorted["week"] == WEEK)
if not mask_cur.any():
    raise ValueError(f"No KPI row found for YEAR={YEAR}, WEEK={WEEK}. Check flagged_weeks selection.")

cur_idx = kpi_weekly_sorted.index[mask_cur][0]
cur_row = kpi_weekly_sorted.iloc[cur_idx]

# Choose comparison: prior week if exists, else baseline
if cur_idx > 0:
    comp_label = "vs prior week"
    comp_row = kpi_weekly_sorted.iloc[cur_idx - 1]
    rev_pc = pct_change(cur_row.revenue, comp_row.revenue)
    inv_pc = pct_change(cur_row.inventory_value, comp_row.inventory_value)
else:
    comp_label = "vs baseline avg"
    rev_pc = pct_change(cur_row.revenue, kpi_weekly_sorted["revenue"].mean())
    inv_pc = pct_change(cur_row.inventory_value, kpi_weekly_sorted["inventory_value"].mean())

top_rev = rev_drivers.iloc[0]
top_inv = inv_drivers.iloc[0]

lines = []
lines.append(f"Week {int(cur_row.week)} Executive Summary (Automotive Parts)")
lines.append(f"• Revenue: ${cur_row.revenue:,.0f} ({rev_pc:+.1f}% {comp_label})" if rev_pc is not None else f"• Revenue: ${cur_row.revenue:,.0f}")
lines.append(f"• Inventory value: ${cur_row.inventory_value:,.0f} ({inv_pc:+.1f}% {comp_label})" if inv_pc is not None else f"• Inventory value: ${cur_row.inventory_value:,.0f}")
if pd.notna(cur_row.abs_forecast_error_pct):
    lines.append(f"• Forecast error (abs %): {cur_row.abs_forecast_error_pct:.2f}%")

lines.append("\nKey drivers:")
lines.append(f"• Biggest revenue change driver: {top_rev.product_category} in {top_rev.region} (Δ ${top_rev.rev_delta:,.0f})")
lines.append(f"• Biggest inventory change driver: SKU {top_inv.sku} at {top_inv.location} (Δ ${top_inv.inv_delta:,.0f})")

lines.append("\nRecommended actions (v0):")
if inv_pc is not None and inv_pc > 10:
    lines.append("• Slow replenishment for top-increasing SKUs/locations; review MOQ, safety stock, and inbound POs.")
if rev_pc is not None and rev_pc < -5:
    lines.append("• Investigate demand drop by region/category; consider promo, channel rebalancing, or price adjustment.")
if pd.notna(cur_row.abs_forecast_error_pct) and cur_row.abs_forecast_error_pct > 15:
    lines.append("• Recalibrate forecast drivers (seasonality/regional lift) and tighten exception thresholds.")

print("\n".join(lines))



Week 1 Executive Summary (Automotive Parts)
• Revenue: $69,973 (-56.7% vs baseline avg)
• Inventory value: $5,419,930 (-51.3% vs baseline avg)
• Forecast error (abs %): 7.26%

Key drivers:
• Biggest revenue change driver: Rotors in Northeast (Δ $4,951)
• Biggest inventory change driver: SKU SUST006 at PHL-DC (Δ $347,453)

Recommended actions (v0):
• Investigate demand drop by region/category; consider promo, channel rebalancing, or price adjustment.
